# Preparation

In [1]:
sILECPath = "/workspace/Projects/ILEC/VBT/Data/ilecdata_20240119"
nPolicyCensusSize = 2000000
arrIssueYearRange = range(2041,2055)

fLowFaceFactorTrue = 0.9
fMaleFactorTrue = 1.2

import polars as pl
import pandas as pd
import pyarrow as pa
import pyarrow.dataset as ds
import numpy as np

import pyvinecopulib as vc
from dateutil.relativedelta import *
from dateutil.rrule import *
from dateutil.parser import *
from datetime import *

import scipy.stats as sps

import os

import dask.dataframe as dd

rng_seed = 0xBEEF
rng = np.random.default_rng(rng_seed)

studyEndDate = datetime.strptime("2053-12-31", "%Y-%m-%d")

/usr/local/lib/python3.10/dist-packages/dask/dataframe/_pyarrow_compat.py:17: FutureWarning: Minimal version of pyarrow will soon be increased to 14.0.1. You are using 14.0.1.dev0+gba5374836.d20240125. Please consider upgrading.
  warnings.warn(


# Source Distribution from ILEC

In [2]:
ilec_data = ds.dataset(source=sILECPath,format='parquet')
  
src_distribution = ilec_data.to_table(filter=(ds.field('Insurance_Plan') == 'Term') & \
    (ds.field('Duration')==1) & \
    (ds.field('Issue_Age') >= 18) & \
    (ds.field('Issue_Age') <= 70) & \
    (ds.field('Number_of_Pfd_Classes')=='4') & \
    (ds.field('Observation_Year') >= 2011) & \
    (ds.field('Observation_Year') <= 2017),\
    columns=['Sex','Issue_Age','Face_Amount_Band','Policies_Exposed'])

src_distribution = pl.from_arrow(src_distribution)
#src_distribution = src_distribution.to_pandas()

FaceGroupMap = {
    '01: 0 - 9,999' : 'Low_Face',
    '02: 10,000 - 24,999' : 'Low_Face',
    '03: 25,000 - 49,999' : 'Low_Face',
    '04: 50,000 - 99,999' : 'Low_Face',
    '05: 100,000 - 249,999' : 'Low_Face',
    '06: 250,000 - 499,999' : 'Low_Face',
    '07: 500,000 - 999,999' : 'High_Face',
    '08: 1,000,000 - 2,499,999' : 'High_Face',
    '09: 2,500,000 - 4,999,999' : 'High_Face',
    '10: 5,000,000 - 9,999,999' : 'High_Face',
    '11: 10,000,000+' : 'High_Face'}

#src_distribution['Face_Group'] = src_distribution['Face_Amount_Band'].map(FaceGroupMap)
#src_distribution.drop('Face_Amount_Band',axis=1,inplace=T)
src_distribution = src_distribution.with_columns(
    Face_Group=src_distribution['Face_Amount_Band'].replace(FaceGroupMap)
)
src_distribution.drop_in_place('Face_Amount_Band')

src_distribution = src_distribution.group_by(['Sex','Issue_Age','Face_Group']).sum().filter(pl.col('Policies_Exposed')>0)

issue_year_proportions = pl.DataFrame({'Issue_Year':arrIssueYearRange,'Proportion':np.linspace(start=.8,stop=1.3,num=len(arrIssueYearRange))})

src_distribution = src_distribution.join(issue_year_proportions,how="cross")

src_distribution = src_distribution.with_columns(
    Policies_Exposed=src_distribution['Proportion']*src_distribution['Policies_Exposed']
)
lostcol = src_distribution.drop_in_place('Proportion')

In [3]:
src_distribution

Sex,Issue_Age,Face_Group,Policies_Exposed,Issue_Year
str,i32,str,f64,i64
"""M""",21,"""Low_Face""",4372.671388,2041
"""M""",21,"""Low_Face""",4582.895974,2042
"""M""",21,"""Low_Face""",4793.12056,2043
"""M""",21,"""Low_Face""",5003.345146,2044
"""M""",21,"""Low_Face""",5213.569732,2045
…,…,…,…,…
"""F""",38,"""Low_Face""",29953.921691,2050
"""F""",38,"""Low_Face""",30959.086848,2051
"""F""",38,"""Low_Face""",31964.252006,2052


In [4]:
policy_pop = src_distribution[
    np.random.choice(a=range(src_distribution.shape[0]),size=nPolicyCensusSize, replace=True, p=src_distribution['Policies_Exposed']/src_distribution['Policies_Exposed'].sum())
]
policy_pop.drop_in_place('Policies_Exposed')

issue_days = pl.DataFrame(
    {'Month'   : range(1,13),
     'MaxDays' : [31,28,31,30,31,30,31,31,30,31,30,31],
     'Proportion' : np.repeat(1/12,12),
     'key' : np.repeat(0,12)}
  ).join(
    pl.DataFrame(
      {
        'Day': range(1,32),
        'key': np.repeat(0,31)
      }
    ),
    how="cross"
  )
  
issue_days = issue_days.filter(pl.col('Day') <= pl.col('MaxDays'))

def prop_map(month):
    match month:
        case 1:
            return .5
        case 12:
            return 1/.5
        case 2:
            return .7
        case 11:
            return 1/.7
        case _:
            return 1

issue_days = issue_days.with_columns(
    Proportion = issue_days['Month'].map_elements(prop_map,return_dtype=float)*issue_days['Proportion']
).with_columns(
    Proportion=issue_days['Proportion']/issue_days['Proportion'].sum()
).drop(['key','key_right'])

issue_days_sample = issue_days[
    np.random.choice(a=range(issue_days.shape[0]),size=nPolicyCensusSize,replace=True,p=issue_days['Proportion'])][['Month','Day']
    ]

policy_pop = pl.concat([policy_pop,issue_days_sample],how="horizontal")

del issue_days_sample

policy_pop = policy_pop.with_columns(
    pl.struct(['Issue_Year','Month','Day']).map_elements(lambda x: date(x['Issue_Year'],x['Month'],x['Day']),return_dtype=date).alias('Issue_Date')
).drop(['Month','Day'])

policy_pop = policy_pop.with_columns(
    Face_Low = np.random.choice(np.arange(start=100,stop=450,step=50)*1000,size=policy_pop.shape[0]),
    Face_High = np.random.choice(np.arange(start=500,stop=2000,step=100)*1000,size=policy_pop.shape[0])
).with_columns(
    pl.struct(['Face_Low','Face_High','Face_Group']).map_elements(lambda x: x['Face_Low'] if x['Face_Group'] == "Low_Face" else x['Face_High'],return_dtype=int).alias('Face_Amount')
).drop(['Face_Low','Face_High'])

policy_pop = policy_pop.with_columns(pl.arange(1, policy_pop.height+1, eager=True).alias("ID"))


In [5]:
policy_pop

Sex,Issue_Age,Face_Group,Issue_Year,Issue_Date,Face_Amount,ID
str,i32,str,i64,date,i64,i64
"""M""",24,"""High_Face""",2046,2046-02-05,1000000,1
"""M""",37,"""High_Face""",2046,2046-11-17,900000,2
"""M""",33,"""Low_Face""",2047,2047-02-18,200000,3
"""M""",45,"""Low_Face""",2054,2054-09-24,400000,4
"""F""",32,"""High_Face""",2050,2050-06-30,1300000,5
…,…,…,…,…,…,…
"""M""",56,"""Low_Face""",2053,2053-07-05,100000,1999996
"""M""",43,"""Low_Face""",2042,2042-04-20,250000,1999997
"""M""",61,"""High_Face""",2042,2042-09-29,1800000,1999998


# Simulate Preferred Characteristics

In [6]:
pcs = [[vc.Bicop(vc.BicopFamily.bb7,180,[1.5,3]),vc.Bicop(vc.BicopFamily.bb7,180,[1.5,3])],[vc.Bicop(vc.BicopFamily.gaussian,0,[.4])]]

vine_cop = vc.Vinecop(vc.CVineStructure([1,2,3]),pair_copulas=pcs)

pref_samples = vine_cop.simulate(n=policy_pop.shape[0],num_threads=8)

pref_samples = pl.DataFrame(data=pref_samples,schema=['OilViscosity_u','OilPressure_u','BMI_u'])

policy_pop = pl.concat([policy_pop,pref_samples],how="horizontal")

In [7]:
policy_pop

Sex,Issue_Age,Face_Group,Issue_Year,Issue_Date,Face_Amount,ID,OilViscosity_u,OilPressure_u,BMI_u
str,i32,str,i64,date,i64,i64,f64,f64,f64
"""M""",24,"""High_Face""",2046,2046-02-05,1000000,1,0.871517,0.879385,0.786948
"""M""",37,"""High_Face""",2046,2046-11-17,900000,2,0.949399,0.961568,0.966345
"""M""",33,"""Low_Face""",2047,2047-02-18,200000,3,0.170633,0.266245,0.305878
"""M""",45,"""Low_Face""",2054,2054-09-24,400000,4,0.135944,0.238985,0.421389
"""F""",32,"""High_Face""",2050,2050-06-30,1300000,5,0.760449,0.276171,0.695298
…,…,…,…,…,…,…,…,…,…
"""M""",56,"""Low_Face""",2053,2053-07-05,100000,1999996,0.334736,0.547832,0.55036
"""M""",43,"""Low_Face""",2042,2042-04-20,250000,1999997,0.438224,0.166023,0.268353
"""M""",61,"""High_Face""",2042,2042-09-29,1800000,1999998,0.128491,0.496163,0.111338


# Convert Simulated Uniform Marginals to Native Scale

In [8]:
def convert_marginal_to_gamma(q, mean=1, variance=1,rounding=2):
  shape = mean*mean/variance
  scale = variance/mean
  
  return round(sps.gamma.ppf(q,a=shape,scale=scale),rounding)

convert_marginal_to_gamma_vec = np.vectorize(convert_marginal_to_gamma)

policy_pop = policy_pop.with_columns(
    pl.struct(['Sex','Face_Group','Issue_Year']).map_elements(
        lambda x: (fMaleFactorTrue if x['Sex'] == 'M' else 1)*(fLowFaceFactorTrue if x['Face_Group'] == 'Low_Face' else 1)*(1+(max(0,x['Issue_Year']-2045))/100),
        return_dtype=float
    ).alias('pref_mean_factor')
).with_columns(
    pl.struct(['BMI_u','pref_mean_factor']).map_elements(
        lambda x: convert_marginal_to_gamma(x['BMI_u'],25*x['pref_mean_factor'],50),
        return_dtype=float
    ).alias('BMI')
).with_columns(
    pl.struct(['OilPressure_u','pref_mean_factor']).map_elements(
        lambda x: convert_marginal_to_gamma(x['OilPressure_u'],30*x['pref_mean_factor'],20),
        return_dtype=float
    ).alias('OilPressure')
).with_columns(
    pl.struct(['OilViscosity_u','pref_mean_factor']).map_elements(
        lambda x: convert_marginal_to_gamma(x['OilViscosity_u'],25*x['pref_mean_factor'],20),
        return_dtype=float
    ).alias('OilViscosity')
)

policy_pop

Sex,Issue_Age,Face_Group,Issue_Year,Issue_Date,Face_Amount,ID,OilViscosity_u,OilPressure_u,BMI_u,pref_mean_factor,BMI,OilPressure,OilViscosity
str,i32,str,i64,date,i64,i64,f64,f64,f64,f64,f64,f64,f64
"""M""",24,"""High_Face""",2046,2046-02-05,1000000,1,0.871517,0.879385,0.786948,1.212,35.67,41.66,35.41
"""M""",37,"""High_Face""",2046,2046-11-17,900000,2,0.949399,0.961568,0.966345,1.212,44.45,44.65,37.98
"""M""",33,"""Low_Face""",2047,2047-02-18,200000,3,0.170633,0.266245,0.305878,1.1016,23.55,30.14,23.28
"""M""",45,"""Low_Face""",2054,2054-09-24,400000,4,0.135944,0.238985,0.421389,1.1772,27.5,32.06,24.58
"""F""",32,"""High_Face""",2050,2050-06-30,1300000,5,0.760449,0.276171,0.695298,1.05,29.35,28.72,29.27
…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""M""",56,"""Low_Face""",2053,2053-07-05,100000,1999996,0.334736,0.547832,0.55036,1.1664,29.48,35.34,27.07
"""M""",43,"""Low_Face""",2042,2042-04-20,250000,1999997,0.438224,0.166023,0.268353,1.08,22.31,28.06,26.07
"""M""",61,"""High_Face""",2042,2042-09-29,1800000,1999998,0.128491,0.496163,0.111338,1.2,21.72,35.77,25.01


In [9]:
# Expensive computation, save the work
policy_pop.write_parquet(file=os.getcwd() + "/policy_pop.parquet")

In [10]:
policy_pop = pl.read_parquet(source=os.getcwd() + "/policy_pop.parquet")
policy_pop = policy_pop.with_columns(pl.arange(1, policy_pop.height+1, eager=True).alias("ID"))

# Apply Mortality Factors for Preferred Factors

In [11]:
def softplus(x):
    return np.log(1+np.exp(x))

policy_pop = policy_pop.with_columns(
    PrefFactor_BMI=np.exp(np.log(1.25)*(pl.col('BMI')-25)/5),
    PrefFactor_Pressure = (np.exp(np.log(1.25)*softplus(.25*(pl.col('OilPressure')-20))) + np.exp(np.log(1.05)*softplus(-1*(pl.col('OilPressure') - 20))))/(1.05+1.25),
    PrefFactor_Viscosity = (np.exp(np.log(1.25)*softplus(.6*(pl.col('OilViscosity')-30))) + np.exp(np.log(1.05)*softplus(-2*(pl.col('OilViscosity') - 30))))/(1.05+1.25)
)

In [12]:
# Normalize

policy_pop = policy_pop.with_columns(
    PrefFactor_Pressure=policy_pop['PrefFactor_Pressure']/policy_pop['PrefFactor_Pressure'].mean(),
    PrefFactor_Viscosity=policy_pop['PrefFactor_Viscosity']/policy_pop['PrefFactor_Viscosity'].mean()
)

In [13]:
# Combine
policy_pop = policy_pop.with_columns(
    PrefFactor = pl.col('PrefFactor_BMI')*pl.col('PrefFactor_Viscosity')*pl.col('PrefFactor_Pressure')
)

# Define Risk Classes and UW Decisions

In [14]:
policy_pop = policy_pop.with_columns(
    UW_Decision=pl.lit("SUB")
).with_columns(
    pl.when(pl.col('BMI').le(41).and_(
        pl.col('OilPressure').le(41),
        pl.col('OilViscosity').le(32)
    )).then(pl.lit("STD")).otherwise(pl.col('UW_Decision')).alias('UW_Decision')
).with_columns(
    pl.when(
        pl.col('PrefFactor').gt(3).and_(pl.col('UW_Decision').eq('SUB'))
    ).then(pl.lit('DEC')).otherwise('UW_Decision').alias('UW_Decision')
)

In [15]:
policy_pop = policy_pop.with_columns(
    pl.lit(3).alias('PrefClass')
).with_columns(
    pl.when(pl.col('BMI').le(30).and_(
                pl.col('OilPressure').le(35),
                pl.col('OilViscosity').le(25),
                pl.col('UW_Decision').eq(pl.lit('STD'))
            )
           ).then(1).otherwise(pl.col('PrefClass')).alias('PrefClass')
).with_columns(
    pl.when(pl.col('BMI').le(33).and_(
                pl.col('OilPressure').le(38),
                pl.col('OilViscosity').le(28),
                pl.col('UW_Decision').eq(pl.lit('STD')),
                pl.col('PrefClass').gt(1)
            )
           ).then(2).otherwise(pl.col('PrefClass')).alias('PrefClass')
)

In [16]:
policy_pop

Sex,Issue_Age,Face_Group,Issue_Year,Issue_Date,Face_Amount,ID,OilViscosity_u,OilPressure_u,BMI_u,pref_mean_factor,BMI,OilPressure,OilViscosity,PrefFactor_BMI,PrefFactor_Pressure,PrefFactor_Viscosity,PrefFactor,UW_Decision,PrefClass
str,i32,str,i64,date,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,i32
"""M""",24,"""High_Face""",2046,2046-02-05,1000000,1,0.871517,0.879385,0.786948,1.212,35.67,41.66,35.41,1.609926,1.351705,1.080649,2.35165,"""SUB""",3
"""M""",37,"""High_Face""",2046,2046-11-17,900000,2,0.949399,0.961568,0.966345,1.212,44.45,44.65,37.98,2.382209,1.540034,1.373575,5.03921,"""DEC""",3
"""M""",33,"""Low_Face""",2047,2047-02-18,200000,3,0.170633,0.266245,0.305878,1.1016,23.55,30.14,23.28,0.937338,0.866984,1.027863,0.8353,"""STD""",1
"""M""",45,"""Low_Face""",2054,2054-09-24,400000,4,0.135944,0.238985,0.421389,1.1772,27.5,32.06,24.58,1.118034,0.925969,0.948963,0.982428,"""STD""",1
"""F""",32,"""High_Face""",2050,2050-06-30,1300000,5,0.760449,0.276171,0.695298,1.05,29.35,28.72,29.27,1.21426,0.828163,0.772467,0.776797,"""STD""",3
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""M""",56,"""Low_Face""",2053,2053-07-05,100000,1999996,0.334736,0.547832,0.55036,1.1664,29.48,35.34,27.07,1.221325,1.045159,0.830318,1.059883,"""STD""",2
"""M""",43,"""Low_Face""",2042,2042-04-20,250000,1999997,0.438224,0.166023,0.268353,1.08,22.31,28.06,26.07,0.886875,0.811472,0.872589,0.627979,"""STD""",2
"""M""",61,"""High_Face""",2042,2042-09-29,1800000,1999998,0.128491,0.496163,0.111338,1.2,21.72,35.77,25.01,0.863828,1.062629,0.925366,0.84942,"""STD""",2


# Sample deaths and lapses

## Deaths

### Load the mortality table and define function for sampling death duration

In [17]:
vbt2015su_m = pd.read_excel(
    os.getcwd() + "/Assumptions/t3265.xlsx",
    skiprows=23,
    nrows=95-18+1
).rename(
    columns={'Row\Column':'Issue_Age'}
).melt(
    id_vars='Issue_Age'
).rename(
    columns={'variable' : 'Duration', 'value':'qx'}
)

vbt2015su_m['Sex'] = 'M'
vbt2015su_m = vbt2015su_m[vbt2015su_m['qx'].notna()]

In [18]:
vbt2015su_f = pd.read_excel(
    os.getcwd() + "/Assumptions/t3266.xlsx",
    skiprows=23,
    nrows=95-18+1
).rename(
    columns={'Row\Column':'Issue_Age'}
).melt(
    id_vars='Issue_Age'
).rename(
    columns={'variable' : 'Duration', 'value':'qx'}
)

vbt2015su_f['Sex'] = 'F'
vbt2015su_f = vbt2015su_f[vbt2015su_f['qx'].notna()]

In [19]:
vbt2015su = pd.concat([vbt2015su_m,vbt2015su_f])
vbt2015su.reset_index(inplace=True,drop=True)
vbt2015su = vbt2015su.astype({'Issue_Age':int,'Duration':int,'qx':float,'Sex':str})

In [20]:
vbt2015ult_m = pd.read_excel(
    os.getcwd() + "/Assumptions/t3265.xlsx",
    skiprows=115,
    usecols=[0,1]
).rename(
    columns={'Row\Column' : 'Attained_Age', '1': 'qx'}
)

vbt2015ult_m['Sex'] = 'M'

In [21]:
vbt2015ult_f = pd.read_excel(
    os.getcwd() + "/Assumptions/t3266.xlsx",
    skiprows=115,
    usecols=[0,1]
).rename(
    columns={'Row\Column' : 'Attained_Age', '1': 'qx'}
)

vbt2015ult_f['Sex'] = 'F'

In [22]:
vbt2015ult = pd.concat([vbt2015ult_m,vbt2015ult_f])
vbt2015ult = vbt2015ult.astype({'Sex':str,'Attained_Age':int,'qx':float})

In [23]:
vbt2015ult

,Attained_Age,qx,Sex
0,18,0.00069,M
1,19,0.00072,M
2,20,0.00074,M
3,21,0.00077,M
4,22,0.00075,M
...,...,...,...
98,116,0.50000,F
99,117,0.50000,F
100,118,0.50000,F
101,119,0.50000,F


In [24]:
vbt2015 = pd.DataFrame(
    data=[(x,y,d) for x in ['M','F'] for y in range(18,96) for d in range(1,121)],
    columns=['Sex','Issue_Age','Duration']
)

vbt2015['Attained_Age'] = vbt2015['Issue_Age'] + vbt2015['Duration'] - 1

vbt2015 = vbt2015[vbt2015['Attained_Age'] <= 120]

In [25]:
vbt2015 = vbt2015.merge(
    vbt2015su,
    how='left',
    on=['Sex','Issue_Age','Duration']
)

vbt2015 = vbt2015.merge(
    vbt2015ult,
    how='left',
    on=['Sex','Attained_Age']
)

vbt2015 = vbt2015.rename(columns={'qx_x' : 'qx_sel', 'qx_y' : 'qx_ult'})

In [26]:
vbt2015

,Sex,Issue_Age,Duration,Attained_Age,qx_sel,qx_ult
0,M,18,1,18,0.00069,0.00069
1,M,18,2,19,0.00072,0.00072
2,M,18,3,20,0.00074,0.00074
3,M,18,4,21,0.00077,0.00077
4,M,18,5,22,0.00075,0.00075
...,...,...,...,...,...,...
10057,F,95,22,116,0.50000,0.50000
10058,F,95,23,117,0.50000,0.50000
10059,F,95,24,118,0.50000,0.50000
10060,F,95,25,119,0.50000,0.50000


In [27]:
vbt2015['qx_su'] = vbt2015.apply(lambda x: x.qx_ult if pd.isnull(x.qx_sel) else x.qx_sel,axis=1)
vbt2015.loc[vbt2015['Attained_Age'] == 120,'qx_ult'] = 1
vbt2015.loc[vbt2015['Attained_Age'] == 120,'qx_su'] = 1

In [28]:
vbt2015

,Sex,Issue_Age,Duration,Attained_Age,qx_sel,qx_ult,qx_su
0,M,18,1,18,0.00069,0.00069,0.00069
1,M,18,2,19,0.00072,0.00072,0.00072
2,M,18,3,20,0.00074,0.00074,0.00074
3,M,18,4,21,0.00077,0.00077,0.00077
4,M,18,5,22,0.00075,0.00075,0.00075
...,...,...,...,...,...,...,...
10057,F,95,22,116,0.50000,0.50000,0.50000
10058,F,95,23,117,0.50000,0.50000,0.50000
10059,F,95,24,118,0.50000,0.50000,0.50000
10060,F,95,25,119,0.50000,0.50000,0.50000


In [29]:
# Define "true" mortality
mort_true = vbt2015.copy()
mort_true['qx_true'] = mort_true.apply(lambda x: x.qx_su * \
                (.9 if x.Sex == 'M' else .95)* \
                (np.power(1.005, x.Duration - 1) if x.Duration <= 20 else np.power(1.005, 19) ), \
                axis=1
               )

In [30]:
mort_true

,Sex,Issue_Age,Duration,Attained_Age,qx_sel,qx_ult,qx_su,qx_true
0,M,18,1,18,0.00069,0.00069,0.00069,0.000621
1,M,18,2,19,0.00072,0.00072,0.00072,0.000651
2,M,18,3,20,0.00074,0.00074,0.00074,0.000673
3,M,18,4,21,0.00077,0.00077,0.00077,0.000703
4,M,18,5,22,0.00075,0.00075,0.00075,0.000689
...,...,...,...,...,...,...,...,...
10057,F,95,22,116,0.50000,0.50000,0.50000,0.522214
10058,F,95,23,117,0.50000,0.50000,0.50000,0.522214
10059,F,95,24,118,0.50000,0.50000,0.50000,0.522214
10060,F,95,25,119,0.50000,0.50000,0.50000,0.522214


In [31]:
def get_durational_distribution(gender, age, issue_year, mort_factor=1):

    df = mort_true[
        (mort_true['Sex'] == gender) &
        (mort_true['Issue_Age'] == age)
        ][['Duration','qx_true']]
    
    df['Calendar_Year'] = issue_year + df['Duration'] - 1
    df.astype({'Calendar_Year':int})
    
    df['qx_true'] = np.fmin(1,df['qx_true']*mort_factor*df.apply(
        lambda x: np.power(1.01, np.fmax(0,np.fmin(x.Calendar_Year - 2045,20))),
        axis=1
    ))
    
    df['tpx_true'] = np.cumprod(1-df['qx_true'])
    df['q_xpt_true'] = df['qx_true']*df['tpx_true'].shift(1,fill_value=1)
    df['q_xpt_true'] = df['q_xpt_true']/df['q_xpt_true'].sum()
    
    return df['q_xpt_true'].to_numpy()

def sample_death_duration(gender, age, issue_year, mort_factor=1, row_index = 1):
    dist = get_durational_distribution(gender,age,issue_year,mort_factor)
    
    np.random.seed(rng_seed + row_index)
    
    return np.random.choice(a=dist.shape[0], p=dist) + 1

In [32]:
policy_pop = policy_pop.with_columns(
    pl.struct(['Sex','Issue_Age','Issue_Year','PrefFactor']).map_elements(lambda x: get_durational_distribution(x['Sex'],x['Issue_Age'],x['Issue_Year'],x['PrefFactor']),return_dtype=object).alias("Dist")
)

In [33]:
policy_pop = policy_pop.with_columns(
    pl.col('Dist').map_elements(lambda x: np.random.choice(a=len(x),p=x)+1,return_dtype=int).alias('Death_Duration')
)

## Lapses

In [34]:
true_lapse = pd.concat(
    [pd.DataFrame(
        {
            'Face_Group' : np.repeat('Low_Face',103),
            'Duration'   : range(1,104),
            'Lapse_Rate' : np.concatenate([np.array([.2,.15,.05,.05,.05]),np.repeat(.03,97),np.array([1])])
        }
      ),
    pd.DataFrame(
    {
        'Face_Group' : np.repeat('High_Face',103),
        'Duration'   : range(1,104),
        'Lapse_Rate' : np.concatenate([np.array([.1,.07,.05,.05,.05]),np.repeat(.03,97),np.array([1])])
    }
    )]
).sort_values(by=['Face_Group','Duration'])

In [35]:
true_lapse['Persistency_Rate'] = 1 - true_lapse['Lapse_Rate']
true_lapse['tpx_lapse'] = true_lapse.groupby('Face_Group')['Persistency_Rate'].cumprod()
true_lapse['w_xpt'] = true_lapse['Lapse_Rate']*true_lapse.groupby('Face_Group')['tpx_lapse'].shift(1,fill_value=1)

In [36]:
policy_pop = policy_pop.with_columns(
    pl.col('Face_Group').map_elements(lambda x: np.random.choice(103,p=true_lapse.loc[true_lapse['Face_Group'] == x,'w_xpt'])+1,return_dtype=int).alias('Lapse_Duration')
)

# Auxiliary Items for Determining Termination Date and Cause

In [37]:
death_skew = (1+np.arange(start=1,stop=366)*.1/364-.05-.1/364)/365

policy_pop = policy_pop.with_columns(
    Prem_Mode = np.random.choice(a=['A','Q','M'],p=[.04,.01,.95],size=nPolicyCensusSize),
    Death_Skew_Days = np.random.choice(a=365,size=nPolicyCensusSize,p=death_skew)+1,
    Lapse_Skew_Months = np.random.choice(a=12,size=nPolicyCensusSize)
)

# Termination Date Computations

In [38]:
def lapse_skew_transform(prem_mode, months):
    match prem_mode:
        case "A":
            return 0
        case "M":
            return months
        case "Q":
            return (months % 4 + 1)*3
        case _:
            return 0

In [39]:
policy_pop = policy_pop.with_columns(
    pl.struct(['Issue_Date','Death_Duration','Death_Skew_Days']).map_elements(lambda x: x['Issue_Date'] + relativedelta(years=x['Death_Duration'] - 1,days=x['Death_Skew_Days']-1),return_dtype=pl.Date).alias('Death_Date')
).with_columns(
    pl.struct(['Issue_Date','Lapse_Duration','Prem_Mode','Lapse_Skew_Months']).map_elements(
        lambda x: x['Issue_Date'] + relativedelta(years=x['Lapse_Duration'] - 1, months=lapse_skew_transform(x['Prem_Mode'],x['Lapse_Skew_Months']))
    ,return_dtype=pl.Date).alias('Lapse_Date')
)

In [40]:
policy_pop = policy_pop.with_columns(
    pl.when(pl.col('UW_Decision').eq(pl.lit('DEC'))).then(
        pl.lit('Not Issued')
    ).when(
        pl.col('Death_Date').le(pl.col('Lapse_Date')).and_(pl.col('Death_Date').le(studyEndDate))
    ).then(pl.lit('Death')).when(
        pl.col('Lapse_Date').le(pl.col('Death_Date')).and_(pl.col('Lapse_Date').le(studyEndDate))
    ).then(pl.lit('Lapsed')).otherwise(pl.lit('Active')).alias('Status')
)

In [41]:
policy_pop.drop_in_place('Dist')
policy_pop.write_parquet(file=os.getcwd() + "/policy_pop.parquet")

In [42]:
policy_pop

Sex,Issue_Age,Face_Group,Issue_Year,Issue_Date,Face_Amount,ID,OilViscosity_u,OilPressure_u,BMI_u,pref_mean_factor,BMI,OilPressure,OilViscosity,PrefFactor_BMI,PrefFactor_Pressure,PrefFactor_Viscosity,PrefFactor,UW_Decision,PrefClass,Death_Duration,Lapse_Duration,Prem_Mode,Death_Skew_Days,Lapse_Skew_Months,Death_Date,Lapse_Date,Status
str,i32,str,i64,date,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,i32,i64,i64,str,i64,i64,date,date,str
"""M""",24,"""High_Face""",2046,2046-02-05,1000000,1,0.871517,0.879385,0.786948,1.212,35.67,41.66,35.41,1.609926,1.351705,1.080649,2.35165,"""SUB""",3,42,28,"""M""",3,9,2087-02-07,2073-11-05,"""Active"""
"""M""",37,"""High_Face""",2046,2046-11-17,900000,2,0.949399,0.961568,0.966345,1.212,44.45,44.65,37.98,2.382209,1.540034,1.373575,5.03921,"""DEC""",3,37,103,"""M""",1,6,2082-11-17,2149-05-17,"""Not Issued"""
"""M""",33,"""Low_Face""",2047,2047-02-18,200000,3,0.170633,0.266245,0.305878,1.1016,23.55,30.14,23.28,0.937338,0.866984,1.027863,0.8353,"""STD""",1,58,8,"""M""",211,11,2104-09-15,2055-01-18,"""Active"""
"""M""",45,"""Low_Face""",2054,2054-09-24,400000,4,0.135944,0.238985,0.421389,1.1772,27.5,32.06,24.58,1.118034,0.925969,0.948963,0.982428,"""STD""",1,34,28,"""M""",31,5,2087-10-24,2082-02-24,"""Active"""
"""F""",32,"""High_Face""",2050,2050-06-30,1300000,5,0.760449,0.276171,0.695298,1.05,29.35,28.72,29.27,1.21426,0.828163,0.772467,0.776797,"""STD""",3,74,9,"""M""",286,5,2124-04-10,2058-11-30,"""Active"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""M""",56,"""Low_Face""",2053,2053-07-05,100000,1999996,0.334736,0.547832,0.55036,1.1664,29.48,35.34,27.07,1.221325,1.045159,0.830318,1.059883,"""STD""",2,26,103,"""M""",241,7,2079-03-02,2156-02-05,"""Active"""
"""M""",43,"""Low_Face""",2042,2042-04-20,250000,1999997,0.438224,0.166023,0.268353,1.08,22.31,28.06,26.07,0.886875,0.811472,0.872589,0.627979,"""STD""",2,55,6,"""A""",333,1,2097-03-18,2047-04-20,"""Lapsed"""
"""M""",61,"""High_Face""",2042,2042-09-29,1800000,1999998,0.128491,0.496163,0.111338,1.2,21.72,35.77,25.01,0.863828,1.062629,0.925366,0.84942,"""STD""",2,17,1,"""M""",162,1,2059-03-09,2042-10-29,"""Lapsed"""
